In [1]:
import notebook_functions
reload(notebook_functions)
from notebook_functions import *
import os, pandas as pd, cPickle as pickle
from nolearn.lasagne.visualize import plot_conv_activity, plot_conv_weights, plot_occlusion
from lasagne.layers import get_output, get_output_shape
import theano.tensor as T
import theano
%pylab inline

Couldn't import dot_parser, loading of dot files will not be possible.
Populating the interactive namespace from numpy and matplotlib

Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, CuDNN 4004)

pylab import has clobbered these variables: ['ceil', 'floor']
`%matplotlib` prevents importing * from pylab and numpy


In [19]:
models = ["net.vgg.large.l2.5e3.orthog-norm-dense", 
          "net.vgg.large.l2.5e3.orthog-norm-maxout8", 
          "net.vgg.large.l2.5e3.orthog-norm-maxout8-lr.2.steps"]

In [10]:
val_dir =  "/media/dylan/Science/Kaggle-Data/distracted_drivers/test/"
input_volume_shape = (128, 128)

(128, 128)

In [4]:
def image_gen_from_dir_with_filenames(data_dir, batches, size):
    result = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) 
                  for f in filenames if os.path.splitext(f)[1] == '.jpg']
    image_files = []
    filenames = []
    for file_path in result:
        image_files.append(read_img_file_PIL(file_path, size=input_volume_shape))
        filenames.append(os.path.basename(file_path))
        if len(image_files) == batches:
            X = np.asarray(image_files, dtype=np.float32)
            X = exposure.rescale_intensity(X, out_range=(-1,1))
            yield X, filenames
            image_files = []
            filenames = []
            
    # end result loop, yield any leftover image files
    X = np.asarray(image_files, dtype=np.float32)
    X = exposure.rescale_intensity(X, out_range=(-1,1))
    yield X, filenames

In [5]:
X, filenames = image_gen_from_dir_with_filenames(val_dir, 20, input_volume_shape).next()

# Evaluation Proper

In [11]:
total_probs = []

In [12]:
for dir_name in models:
    validation_file_name = "{}/vloss-{}.txt".format(dir_name, dir_name)
    model_file_name = "{}/{}.pickle".format(dir_name, dir_name)
    best_weights_file_name = "{}/bw-{}.weights".format(dir_name, dir_name)
    print "\n=== Loading model: {} ===\n".format(dir_name)
    with open(model_file_name, 'rb') as reader:
        net = pickle.load(reader)
    load_best_weights(best_weights_file_name, net)
    
    image_gen = image_gen_from_dir_with_filenames(val_dir, 40, input_volume_shape)
    threaded_gen = threaded_generator(image_gen, num_cached=40)

    all_probs = []
    all_file_names = []
    for X, file_names in threaded_gen:
        all_probs.extend(net.predict_proba(X.reshape(-1, 1, input_volume_shape[0], input_volume_shape[1])))
        all_file_names.extend(file_names)
    
    total_probs.append(all_probs)


=== Loading model: net.vgg.large.l2.5e3.orthog-norm-dense ===

Loaded parameters to layer 'conv2ddnn1' (shape 32x1x3x3).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'batchnorm2' (shape 32).
Loaded parameters to layer 'conv2ddnn4' (shape 32x32x3x3).
Loaded parameters to layer 'batchnorm5' (shape 32).
Loaded parameters to layer 'batchnorm5' (shape 32).
Loaded parameters to layer 'batchnorm5' (shape 32).
Loaded parameters to layer 'batchnorm5' (shape 32).
Loaded parameters to layer 'conv2ddnn9' (shape 64x32x3x3).
Loaded parameters to layer 'batchnorm10' (shape 64).
Loaded parameters to layer 'batchnorm10' (shape 64).
Loaded parameters to layer 'batchnorm10' (shape 64).
Loaded parameters to layer 'batchnorm10' (shape 64).
Loaded parameters to layer 'conv2ddnn12' (shape 64x64x3x3).
Loaded parameters to layer 'batchnorm13' (shape 64).
Loaded parameters t

IOError: [Errno 2] No such file or directory: 'net.vgg.large.l2.5e3.orthog-norm-maxout8-lr.2.steps//net.vgg.large.l2.5e3.orthog-norm-maxout8-lr.2.steps/.pickle'

In [24]:
ave_probs = np.mean(total_probs, axis=0)

In [25]:
df = pd.DataFrame(np.asarray(ave_probs), 
                  index = all_file_names, columns=["c0","c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9"])
df.index.name = 'img'
df.shape

(79726, 10)

In [26]:
df.to_csv("preds.bagged.001.csv")

In [27]:
df[:10]

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
img,,,,,,,,,,
img_73540.jpg,0.002068,0.001070,0.001171,0.001879,0.001061,0.988639,0.001038,0.001012,0.000941,0.001121
img_9672.jpg,0.002029,0.002374,0.982668,0.001452,0.001601,0.001600,0.001891,0.001657,0.002499,0.002229
img_49295.jpg,0.001817,0.001031,0.001063,0.001287,0.989370,0.001161,0.001099,0.001106,0.000996,0.001070
img_91517.jpg,0.003012,0.001345,0.001518,0.001606,0.001643,0.985064,0.001751,0.001497,0.001173,0.001393
img_51925.jpg,0.954098,0.003298,0.002645,0.008444,0.004786,0.007253,0.003060,0.002755,0.002484,0.011176
img_87962.jpg,0.000895,0.000900,0.000876,0.000884,0.000913,0.001025,0.000864,0.991558,0.001029,0.001055
img_30198.jpg,0.004367,0.003663,0.001313,0.982495,0.001483,0.001418,0.001506,0.001235,0.001362,0.001158
img_78210.jpg,0.007895,0.009034,0.005388,0.004335,0.004090,0.003929,0.004543,0.004333,0.920090,0.036363
img_19345.jpg,0.001147,0.001282,0.001386,0.002035,0.986630,0.001751,0.001483,0.001299,0.001473,0.001514
